In [19]:
import pandas as pd
from datetime import datetime
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [20]:
tr = pd.read_csv('EDA_transaction.csv')
cu = pd.read_csv('EDA_customer.csv')

# 2016-06-30 데이터 삭제
tr = tr[tr['created_at'] != "2016-06-30 23:18:44.792905+00:00"].reset_index(drop=True)

tr['price'] = tr['item_price'] * tr['quantity']
tr

,created_at,customer_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,total_amount,product_id,quantity,item_price,year,shipment_time,distance(km),buy_age,price
0,2018-07-29 15:22:01.458193+00:00,5868,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03 05:07:24.812676+00:00,199832,44660.0,1,191247,2018,4 days,608.611583,17,191247
1,2018-07-30 12:40:22.365620+00:00,4774,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,Not Promoted,10000,2018-08-03 01:29:03.415705+00:00,155526,16193.0,1,145526,2018,3 days,1577.934908,22,145526
2,2018-09-15 11:51:17.365620+00:00,4774,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,Not Promoted,10000,2018-09-18 08:41:49.422380+00:00,550696,53686.0,4,135174,2018,2 days,525.418925,22,540696
3,2018-11-01 11:23:48.365620+00:00,4774,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,Not Promoted,0,2018-11-05 17:42:27.954235+00:00,271012,20228.0,1,271012,2018,4 days,0.000000,22,271012
4,2018-12-18 11:20:30.365620+00:00,4774,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,Not Promoted,0,2018-12-23 17:24:07.361785+00:00,198753,55220.0,1,198753,2018,5 days,0.000000,22,198753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254579,2021-09-21 22:10:05.074873+00:00,25121,484eecde-9f4a-42de-a8df-7ce6668101f4,Debit Card,Failed,0,Not Promoted,0,2021-09-27 00:43:40.992167+00:00,226820,30642.0,2,113410,2021,5 days,0.000000,28,226820
1254580,2021-12-09 07:35:18.222046+00:00,34829,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12 07:52:44.803822+00:00,359203,19394.0,1,354213,2021,3 days,918.726839,15,354213
1254581,2021-12-11 01:20:36.944593+00:00,99645,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12 13:28:22.848703+00:00,1158159,24250.0,4,288835,2021,1 days,0.000000,30,1155340
1254582,2019-12-18 11:01:09.623382+00:00,87319,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23 22:56:46.831720+00:00,184143,11312.0,1,188646,2019,5 days,0.000000,12,188646


### 모델링에 사용할 특성 엔지니어링

#### 소비 금액 (monetary)

In [21]:
price = tr.groupby('customer_id', as_index=False)['price'].sum()
price

,customer_id,price
0,3,21031971
1,8,3861260
2,9,2603665
3,11,192025
4,15,2101232
...,...,...
50700,99992,4234671
50701,99995,35557417
50702,99997,216461
50703,99998,6667865


In [22]:
standard_date = pd.to_datetime('2022-08-01')

cu['first_join_date'] = pd.to_datetime(cu['first_join_date'])
cu['join_days'] = (standard_date - cu['first_join_date']).astype(str).apply(lambda x: str(x)[:-5]).astype(int)
cu['join_years'] = cu['join_days'] / 365
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863


In [23]:
cu = pd.merge(cu, price, on='customer_id', how='outer').fillna(0)
cu['price_per_year'] = cu['price'] / cu['join_years']
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00


#### 이탈 여부

In [24]:
last_tr = tr.drop_duplicates(subset=['created_at'], keep='last').reset_index(drop=True)
last_tr = last_tr.drop_duplicates(subset=['customer_id'], keep='last').reset_index(drop=True)
last_tr = last_tr[['created_at', 'customer_id']]
last_tr

,created_at,customer_id
0,2018-07-29 15:22:01.458193+00:00,5868
1,2022-06-09 12:18:09.365620+00:00,4774
2,2022-07-08 18:02:01.162570+00:00,58191
3,2018-07-28 21:54:19.891293+00:00,81104
4,2021-03-31 17:25:18.820299+00:00,47040
...,...,...
50700,2021-09-21 22:10:05.074873+00:00,25121
50701,2021-12-09 07:35:18.222046+00:00,34829
50702,2021-12-11 01:20:36.944593+00:00,99645
50703,2019-12-18 11:01:09.623382+00:00,87319


In [25]:
last_tr['last_transaction'] = last_tr['created_at'].apply(lambda x: str(x)[:-22])
last_tr

,created_at,customer_id,last_transaction
0,2018-07-29 15:22:01.458193+00:00,5868,2018-07-29
1,2022-06-09 12:18:09.365620+00:00,4774,2022-06-09
2,2022-07-08 18:02:01.162570+00:00,58191,2022-07-08
3,2018-07-28 21:54:19.891293+00:00,81104,2018-07-28
4,2021-03-31 17:25:18.820299+00:00,47040,2021-03-31
...,...,...,...
50700,2021-09-21 22:10:05.074873+00:00,25121,2021-09-21
50701,2021-12-09 07:35:18.222046+00:00,34829,2021-12-09
50702,2021-12-11 01:20:36.944593+00:00,99645,2021-12-11
50703,2019-12-18 11:01:09.623382+00:00,87319,2019-12-18


In [26]:
cu = pd.merge(cu, last_tr[['customer_id', 'last_transaction']], on='customer_id', how='outer')
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year,last_transaction
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00,NaN
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00,NaN
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07,2022-07-29
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00,NaN
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04,2021-04-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06,2022-07-27
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06,2022-06-29
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07,2022-07-29
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00,NaN


In [27]:
cu.isnull().sum()

customer_id             0
gender                  0
age                     0
join_age                0
device                  0
device_version          0
device_version2         0
home_lat                0
home_long               0
home_location           0
birthdate               0
first_join_date         0
join_days               0
join_years              0
price                   0
price_per_year          0
last_transaction    49295
dtype: int64

In [28]:
# 결측치 처리
cu['last_transaction'] = cu['last_transaction'].fillna(cu['first_join_date'])
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year,last_transaction
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00,2019-07-21 00:00:00
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00,2017-07-16 00:00:00
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07,2022-07-29
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00,2021-10-03 00:00:00
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04,2021-04-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06,2022-07-27
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06,2022-06-29
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07,2022-07-29
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00,2020-05-31 00:00:00


In [29]:
cu['last_transaction'] = cu['last_transaction'].apply(lambda x: str(x)[:10])
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year,last_transaction
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00,2019-07-21
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00,2017-07-16
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07,2022-07-29
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00,2021-10-03
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04,2021-04-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06,2022-07-27
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06,2022-06-29
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07,2022-07-29
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00,2020-05-31


In [30]:
standard_date = pd.to_datetime('2022-08-01')
cu['last_transaction'] = pd.to_datetime(cu['last_transaction'])

cu['last_transaction_days'] = (standard_date - cu['last_transaction']).dt.days
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year,last_transaction,last_transaction_days
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00,2019-07-21,1107
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00,2017-07-16,1842
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07,2022-07-29,3
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00,2021-10-03,302
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04,2021-04-16,472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06,2022-07-27,5
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06,2022-06-29,33
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07,2022-07-29,3
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00,2020-05-31,792


In [31]:
# 이탈 기준 : 120일 → 이탈 여부 컬럼 생성

def churn(row):
    if row['last_transaction_days'] > 120:
        return 'T'  # 이탈
    else:
        return 'F'  # 이탈 X

cu['churn'] = cu.apply(churn, axis=1)
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,birthdate,first_join_date,join_days,join_years,price,price_per_year,last_transaction,last_transaction_days,churn
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,1996-06-14,2019-07-21,1107,3.032877,0.0,0.000000e+00,2019-07-21,1107,T
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,1993-08-16,2017-07-16,1842,5.046575,0.0,0.000000e+00,2017-07-16,1842,T
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,1989-01-23,2020-08-23,708,1.939726,47964122.0,2.472727e+07,2022-07-29,3,F
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,1991-01-05,2021-10-03,302,0.827397,0.0,0.000000e+00,2021-10-03,302,T
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2000-07-15,2021-04-11,477,1.306849,115702.0,8.853507e+04,2021-04-16,472,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2000-11-17,2020-10-17,653,1.789041,15988231.0,8.936760e+06,2022-07-27,5,F
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,1998-03-30,2017-11-25,1710,4.684932,30988098.0,6.614419e+06,2022-06-29,33,F
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,1993-01-12,2018-09-01,1430,3.917808,56830108.0,1.450559e+07,2022-07-29,3,F
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2000-09-13,2020-05-31,792,2.169863,0.0,0.000000e+00,2020-05-31,792,T


In [32]:
# 필요 없는 컬럼 삭제
cu = cu.drop(['price', 'last_transaction', 'last_transaction_days', 'birthdate'], axis=1)
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T


In [33]:
cycle = pd.read_csv('customer_access_cycle.csv')
cycle

,customer_id,mean_access_cycle,median_access_cycle
0,1.0,2072.000000,2072.0
1,2.0,218.000000,218.0
2,3.0,6.310526,1.0
3,4.0,496.000000,496.0
4,5.0,225.000000,225.0
...,...,...,...
99995,99996.0,109.000000,109.0
99996,99997.0,827.000000,827.0
99997,99998.0,15.444444,1.0
99998,99999.0,2118.000000,2118.0


In [34]:
cu = pd.merge(cu, cycle[['customer_id', 'median_access_cycle']], on='customer_id', how='outer')
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0


### promo 관련 컬럼 생성

In [35]:
tr_del = tr.drop_duplicates(subset='created_at')

promo_price = tr_del.groupby('customer_id', as_index=False)['promo_amount'].sum()
promo_price.rename(columns={'promo_amount':'promo_price'}, inplace=True)

cu = pd.merge(cu, promo_price, on='customer_id', how='outer')
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle,promo_price
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0,NaN
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0,NaN
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0,95457.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0,NaN
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0,84542.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0,69897.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0,177520.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0,NaN


In [36]:
promo_count = tr_del.groupby('customer_id', as_index=False)['created_at'].count()
promo_count.rename(columns={'created_at':'promo_count'}, inplace=True)

cu = pd.merge(cu, promo_count, on='customer_id', how='outer')
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle,promo_price,promo_count
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0,NaN,NaN
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0,NaN,NaN
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0,95457.0,89.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0,NaN,NaN
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0,84542.0,37.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0,69897.0,44.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0,177520.0,96.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0,NaN,NaN


In [37]:
# 결측치 처리
cu['promo_price'] = cu['promo_price'].fillna(0)
cu['promo_count'] = cu['promo_count'].fillna(0)
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle,promo_price,promo_count
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0,0.0,0.0
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0,0.0,0.0
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0,95457.0,89.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0,0.0,0.0
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0,84542.0,37.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0,69897.0,44.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0,177520.0,96.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0,0.0,0.0


### 배송비 관련 컬럼 생성

In [39]:
shipment_fee = tr_del.groupby('customer_id', as_index=False)['shipment_fee'].sum()
shipment_fee

cu = pd.merge(cu, shipment_fee, on='customer_id', how='outer')
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle,promo_price,promo_count,shipment_fee
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0,0.0,0.0,NaN
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0,0.0,0.0,NaN
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0,95457.0,89.0,825000.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0,0.0,0.0,NaN
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0,84542.0,37.0,305000.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0,69897.0,44.0,405000.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0,177520.0,96.0,815000.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0,0.0,0.0,NaN


In [40]:
# 결측치 처리
cu['shipment_fee'] = cu['shipment_fee'].fillna(0)
cu

,customer_id,gender,age,join_age,device,device_version,device_version2,home_lat,home_long,home_location,first_join_date,join_days,join_years,price_per_year,churn,median_access_cycle,promo_price,promo_count,shipment_fee
0,2870,F,28,24,iPhone,14.2.1,14,-1.043345,101.360523,Sumatera Barat,2019-07-21,1107,3.032877,0.000000e+00,T,1106.0,0.0,0.0,0.0
1,8193,F,31,25,Android,2.2.1,2,-6.212489,106.818850,Jakarta Raya,2017-07-16,1842,5.046575,0.000000e+00,T,1841.0,0.0,0.0,0.0
2,7279,M,35,32,iPad,4.2.1,4,-8.631607,116.428436,Nusa Tenggara Barat,2020-08-23,708,1.939726,2.472727e+07,F,4.0,95457.0,89.0,825000.0
3,88813,M,33,31,iPad,14.2,14,1.299332,115.774934,Kalimantan Timur,2021-10-03,302,0.827397,0.000000e+00,T,301.0,0.0,0.0,0.0
4,82542,M,24,22,iPhone,10.3.3,10,-2.980807,114.924675,Kalimantan Selatan,2021-04-11,477,1.306849,8.853507e+04,T,471.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,34743,M,24,21,Android,4.3.1,4,-5.203523,105.635530,Lampung,2020-10-17,653,1.789041,8.936760e+06,F,3.0,84542.0,37.0,305000.0
99996,72065,M,26,20,Android,8.0.0,8,4.474702,95.904876,Aceh,2017-11-25,1710,4.684932,6.614419e+06,F,2.0,69897.0,44.0,405000.0
99997,32818,F,31,26,Android,11,11,-3.330376,127.028348,Maluku,2018-09-01,1430,3.917808,1.450559e+07,F,3.0,177520.0,96.0,815000.0
99998,75597,M,24,21,Android,7.1,7,-2.173114,101.495913,Jambi,2020-05-31,792,2.169863,0.000000e+00,T,791.0,0.0,0.0,0.0


In [41]:
# 수정된 customer 파일 변환
cu.to_csv("EDA_customer2.csv", index=False, encoding="utf-8-sig", mode="w")